# 4. Ensembles de Arboles de Decision

## 4.03 Random Forest

*Random Forest* es un algoritmo de ensembles de arboles de decision creado por Leo Brieman en 1995-2006
https://link.springer.com/content/pdf/10.1023/a:1010933404324.pdf

La página original es:
https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm

Dos buenos videos para seguir el paso a paso de Random Forest y aplicaciones:
* https://www.youtube.com/watch?v=J4Wdy0Wc_xQ
* https://www.youtube.com/watch?v=sQ870aTKqiM

Qué tipo de perturbaciones se realizan en Random Forest

*   Se perturba el dataset, con la técnica de bagging = Bootstrap Aggregating
*   Tambien se perturba el algoritmo, utiliza random en cada split

Cada arbolito de Random Forest se entrena sobre un dataset perturbado, que tiene :
* todas las columnas originales (esta es una GRAN diferencia con  Arboles Azarosos)
* la misma *cantidad* de registros del dataset original, PERO generados por la técnica de sampleo con reposición del dataset original.

A pesar de que Leo Brieman es también el inventor de CART (Classification and Regression Trees) Random Forest no corre el algoritmo CART de la libreria rpart, sino un CART perturbado, en donde cada split NO se hace sobre todos los campos del dataset, sino sobre un csubconjunto tomado al azar, esa cantidad es el hiperparámetro *mtry*

#### 4.03.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   Bajar el  **competencia_01_crudo**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_01_crudo.csv"
archivo_destino="/content/datasets/competencia_01_crudo.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/competencia_01_crudo.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


## Generacion de la clase_ternaria

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

In [ ]:
require( "data.table" )

# leo el dataset
dataset <- fread("/content/datasets/competencia_01_crudo.csv" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
    "pos" = .I,
    numero_de_cliente,
    periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 ) ]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]


# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

fwrite( dataset,
    file =  "/content/datasets/competencia_01.csv.gz",
    sep = ","
)

Loading required package: data.table



In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
202101,BAJA+1,622
202101,BAJA+2,825
202101,CONTINUA,160080
202102,BAJA+1,831
202102,BAJA+2,1032
202102,CONTINUA,160292
202103,BAJA+1,1039
202103,BAJA+2,951
202103,CONTINUA,161119


### 4.04  Random Forest, una corrida

El tiempo de corrida de este punto es de alrededor de 8 minutos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Sep 01 08:51:05 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,725735,38.8,1454468,77.7,1454468,77.7
Vcells,1374046,10.5,169515428,1293.4,211569306,1614.2


**ranger** es una de las muchas librerías en lenguage R que implementa el algoritmo *Random Forest*, tiene la ventaja que corre el paralelo, utilizando todos los nucleos del procesador.

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

# ranger se usa para procesar
if( !require("ranger") ) install.packages("ranger")
require("ranger")

# randomForest  solo se usa para imputar nulos
if( !require("randomForest") ) install.packages("randomForest")
require("randomForest")

Loading required package: rpart

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attac

Aqui debe cargar SU semilla primigenia y

In [ ]:
PARAM <- list()
PARAM$experimento <- 440
PARAM$semilla_primigenia <- 102191

# training y future
PARAM$train <- c(202102)
PARAM$future <- c(202104)

PARAM$ranger$num.trees <- 300 # cantidad de arboles
PARAM$ranger$mtry <- 13 # cantidad de atributos que participan en cada split
PARAM$ranger$min.node.size <- 50 # tamaño minimo de las hojas
PARAM$ranger$max.depth <- 10 # 0 significa profundidad infinita


PARAM$semilla_kaggle <- 314159

In [ ]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

In [ ]:
# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )

  return( drealidad )
}

In [ ]:
# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}

In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KA", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/competencia_01.csv.gz")

In [ ]:
#  estas dos lineas estan relacionadas con el Data Drifting
# asigno un valor muy negativo

if( "Master_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Master_Finiciomora) , Master_Finiciomora := -999 ]

if( "Visa_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Visa_Finiciomora) , Visa_Finiciomora :=  -999 ]


In [ ]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes %in% PARAM$train]

In [ ]:
# mes donde voy a aplicar el modelo
dfuture <- dataset[foto_mes %in% PARAM$future]
setorder(dfuture, numero_de_cliente, foto_mes)

In [ ]:
# inicilizo el dataset  drealidad
drealidad <- realidad_inicializar( dfuture, PARAM)

In [ ]:
# quito clase ternaria de donde voy a aplicar el modelo
dfuture[, clase_ternaria:= NULL]

In [ ]:
set.seed(PARAM$semilla_primigenia,"L'Ecuyer-CMRG" ) # Establezco la semilla aleatoria

# ranger necesita la clase de tipo factor
factorizado <- as.factor(dtrain$clase_ternaria)
dtrain[, clase_ternaria := factorizado]

In [ ]:
# Ranger NO acepta valores nulos
# Leo Breiman, ¿por que le temias a los nulos?
# imputo los nulos, ya que ranger no acepta nulos
dtrain <- na.roughfix(dtrain)


In [ ]:
setorder(dtrain, clase_ternaria) # primero quedan los BAJA+1, BAJA+2, CONTINUA

# genero el modelo de Random Forest llamando a ranger()
modelo <- ranger(
  formula= "clase_ternaria ~ .",
  data= dtrain,
  probability= TRUE, # para que devuelva las probabilidades
  num.trees= PARAM$ranger$num.trees,
  mtry= PARAM$ranger$mtry,
  min.node.size= PARAM$ranger$min.node.size,
  max.depth= PARAM$ranger$max.depth
)


Growing trees.. Progress: 16%. Estimated remaining time: 2 minutes, 46 seconds.
Growing trees.. Progress: 32%. Estimated remaining time: 2 minutes, 18 seconds.
Growing trees.. Progress: 48%. Estimated remaining time: 1 minute, 44 seconds.
Growing trees.. Progress: 64%. Estimated remaining time: 1 minute, 12 seconds.
Growing trees.. Progress: 80%. Estimated remaining time: 40 seconds.
Growing trees.. Progress: 96%. Estimated remaining time: 8 seconds.


In [ ]:
# Carpinteria necesaria sobre  dfuture
# como quiere la Estadistica Clasica, imputar nulos por separado
# ( aunque en este caso ya tengo los datos del futuro de antemano
#  pero bueno, sigamos el librito de estos fundamentalistas a rajatabla ...

dfuture <- na.roughfix(dfuture)

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]

In [ ]:
# aplico el modelo a los datos que no tienen clase
# aplico el modelo recien creado a los datos del futuro
prediccion <- predict(modelo, dfuture)

tb_prediccion[, prob := prediccion$predictions[, "BAJA+2"] ]

In [ ]:
tb_prediccion[, Predicted := as.numeric(prob > (1/40))]

In [ ]:
res <- realidad_evaluar( drealidad, tb_prediccion)

In [ ]:
cat( " TOTAL=", res$total,
  " Public=", res$public,
  " Private=", res$private,
  "\n",
  sep= ""
)

 Total=307560000 Public=286266667 Private=316685714


In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

Reportar el resultado  TOTAL en la hoja C4-Random Forest de la planilla colaborativa

Usted NO reportará la ganancia del Public ni tampoco la del Private, por ahora es simplemente para que perciba la variabilidad existente y comience a tener plena conciencia de los fenómenos que observó en la asignatura  Data Mining del cuatrimestre anterior.



---



### 4.05  Random Forest  optimizacion de hiperparámetros

Random Forest es un algoritmo que quedó obsoleto luego de la aparición de  XGBoost y LightGBM, debido a lo lento de las librerías que lo implementan.
<br> El siguiente script se brinda simplemente a modo pedagógico, advirtiendo a l@s alumn@s que demanda más de 24 horas para correr, y los resultados no son brillantes.

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Sep 01 09:18:51 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1944822,103.9,3723718,198.9,3723718,198.9
Vcells,3576386,27.3,191448098,1460.7,239242511,1825.3


**ranger** es una de las muchas librerías en lenguage R que implementa el algoritmo *Random Forest*, tiene la ventaja que corre el paralelo, utilizando todos los nucleos del procesador.

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

# ranger se usa para procesar
if( !require("ranger") ) install.packages("ranger")
require("ranger")

# randomForest  solo se usa para imputar nulos
if( !require("randomForest") ) install.packages("randomForest")
require("randomForest")


if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


Loading required package: parallel

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes

Loading required package: rlist

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rlist’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘XML’


Loading required package: rlist

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

War

Aqui debe cargar SU semilla primigenia y

In [ ]:
PARAM <- list()
PARAM$experimento <- 450
PARAM$semilla_primigenia <- 102191

PARAM$hyperparametertuning$iteraciones <- 40
PARAM$hyperparametertuning$xval_folds <- 5
PARAM$hyperparametertuning$POS_ganancia <- 780000
PARAM$hyperparametertuning$NEG_ganancia <- -20000

# Estructura que define los hiperparámetros y sus rangos
#  la letra L al final significa ENTERO
# max.depth 0 significa profundidad infinita
PARAM$hyperparametertuning$hs <- makeParamSet(
  makeIntegerParam("num.trees", lower= 20L, upper= 500L),
  makeIntegerParam("max.depth", lower= 1L, upper= 30L),
  makeIntegerParam("min.node.size", lower= 1L, upper= 1000L),
  makeIntegerParam("mtry", lower= 2L, upper= 50L)
)

# training
PARAM$train <- c(202102)

In [ ]:
# graba a un archivo los componentes de lista
# para el primer registro, escribe antes los titulos

loguear <- function(
    reg, arch= NA, folder= "./work/",
    ext= ".txt", verbose= TRUE) {

  archivo <- arch
  if (is.na(arch)) archivo <- paste0(folder, substitute(reg), ext)

  if (!file.exists(archivo)) # Escribo los titulos
    {
      linea <- paste0(
        "fecha\t",
        paste(list.names(reg), collapse= "\t"), "\n"
      )

      cat(linea, file= archivo)
    }

  linea <- paste0(
    format(Sys.time(), "%Y%m%d %H%M%S"), "\t", # la fecha y hora
    gsub(", ", "\t", toString(reg)), "\n"
  )

  cat(linea, file= archivo, append= TRUE) # grabo al archivo

  if (verbose) cat(linea) # imprimo por pantalla
}


In [ ]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30
# particionar( data=dataset, division=c(1,1,1,1,1),
#   agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar <- function(
    data, division, agrupa= "",
    campo= "fold", start= 1, seed= NA) {

  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by= agrupa
  ]
}


In [ ]:
# es un paso del Cross Validation
# utiliza el fold  fold_test para testear y el resto para entrenar

ranger_Simple <- function(fold_test, pdata, param) {
  # genero el modelo

  set.seed(PARAM$semillas[2])

  modelo <- ranger(
    formula= "clase_binaria ~ .",
    data= pdata[fold != fold_test],
    probability= TRUE, # para que devuelva las probabilidades
    num.trees= param$num.trees,
    mtry= param$mtry,
    min.node.size= param$min.node.size,
    max.depth= param$max.depth
  )

  prediccion <- predict(modelo, pdata[fold == fold_test])

  ganancia_testing <- pdata[
    fold == fold_test,
    sum((prediccion$predictions[, "POS"] > 1 / 40) *
      ifelse(clase_binaria == "POS",
        PARAM$hyperparametertuning$POS_ganancia,
        PARAM$hyperparametertuning$NEG_ganancia
      ))
  ]

  return(ganancia_testing)
}


In [ ]:
# realiza Cross Validation, promediando las ganancias de los folds de testing

ranger_CrossValidation <- function(
    data, param,
    pcampos_buenos, qfolds, pagrupa, semilla) {

  divi <- rep(1, qfolds)
  particionar(data, divi, seed= semilla, agrupa= pagrupa)

  ganancias <- mcmapply(ranger_Simple,
    seq(qfolds), # 1 2 3 4 5
    MoreArgs= list(data, param),
    SIMPLIFY= FALSE,
    mc.cores= 1
  ) # dejar esto en  1, porque ranger ya corre en paralelo

  data[, fold := NULL] # elimino el campo fold

  # devuelvo la ganancia promedio normalizada
  ganancia_promedio <- mean(unlist(ganancias))
  ganancia_promedio_normalizada <- ganancia_promedio * qfolds

  return(ganancia_promedio_normalizada)
}

In [ ]:
# esta funcion solo puede recibir los parametros que se estan optimizando
# el resto de los parametros se pasan como variables globales

EstimarGanancia_ranger <- function(x) {
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  xval_folds <- PARAM$hyperparametertuning$xval_folds

  ganancia <- ranger_CrossValidation(dataset,
    param= x,
    qfolds= xval_folds,
    pagrupa= "clase_binaria",
    semilla= PARAM$semillas[1]
  )

  # logueo
  xx <- x
  xx$xval_folds <- xval_folds
  xx$ganancia <- ganancia
  xx$iteracion <- GLOBAL_iteracion
  loguear(xx, arch= klog)

  # si es ganancia superadora la almaceno en mejor
  if( ganancia > GLOBAL_mejor ) {
    GLOBAL_mejor <<- ganancia
    loguear(xx, arch= klog_mejor)
  }


  return(ganancia)
}


aqui se inicia el programa

In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [ ]:
# genero numeros primos
primos <- generate_primes(min= 100000, max= 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, 2 )


In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/competencia_01.csv.gz")

In [ ]:
# solo trabajo con  training
dataset <- dataset[foto_mes %in% PARAM$train ]

In [ ]:
#  estas dos lineas estan relacionadas con el Data Drifting
# asigno un valor muy negativo

if( "Master_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Master_Finiciomora) , Master_Finiciomora := -999 ]

if( "Visa_Finiciomora" %in% colnames(dataset) )
  dataset[ is.na(Visa_Finiciomora) , Visa_Finiciomora :=  -999 ]


In [ ]:
set.seed(PARAM$semilla_primigenia,"L'Ecuyer-CMRG" ) # Establezco la semilla aleatoria

In [ ]:
# en estos archivos quedan los resultados
kbayesiana <- paste0("HT", PARAM$experimento, ".RDATA")
klog <- paste0("HT", PARAM$experimento, ".txt")
klog_mejor <- paste0("HT", PARAM$experimento, "_mejor.txt")

GLOBAL_iteracion <- 0 # inicializo la variable global
GLOBAL_mejor <- -Inf

# si ya existe el archivo log, traigo hasta donde llegue
if (file.exists(klog)) {
  tabla_log <- fread(klog)
  GLOBAL_iteracion <- nrow(tabla_log)
}


In [ ]:
# paso a trabajar con clase binaria POS={BAJA+2}   NEG={BAJA+1, CONTINUA}
dataset[, clase_binaria :=
  as.factor(ifelse(clase_ternaria == "BAJA+2", "POS", "NEG"))]

dataset[, clase_ternaria := NULL] # elimino la clase_ternaria, ya no la necesito


In [ ]:
# Ranger NO acepta valores nulos
# Leo Breiman, ¿por que le temias a los nulos?
# imputo los nulos, ya que ranger no acepta nulos

dataset <- na.roughfix(dataset)

In [ ]:
# Aqui comienza la configuracion de la Bayesian Optimization

configureMlr(show.learner.output = FALSE)

funcion_optimizar <- EstimarGanancia_ranger

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar,
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hyperparametertuning$hs,
  has.simple.signature= FALSE
)

ctrl <- makeMBOControl(save.on.disk.at.time= 600, save.file.path= kbayesiana)

ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
)

ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists(kbayesiana)) {
  run <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  run <- mboContinue(kbayesiana)
} # retomo en caso que ya exista

Computing y column(s) for design. Not provided.



Growing trees.. Progress: 16%. Estimated remaining time: 2 minutes, 42 seconds.
Growing trees.. Progress: 33%. Estimated remaining time: 2 minutes, 8 seconds.
Growing trees.. Progress: 49%. Estimated remaining time: 1 minute, 38 seconds.
Growing trees.. Progress: 66%. Estimated remaining time: 1 minute, 5 seconds.
Growing trees.. Progress: 82%. Estimated remaining time: 35 seconds.
Growing trees.. Progress: 98%. Estimated remaining time: 4 seconds.
Growing trees.. Progress: 16%. Estimated remaining time: 2 minutes, 39 seconds.
Growing trees.. Progress: 32%. Estimated remaining time: 2 minutes, 9 seconds.
Growing trees.. Progress: 49%. Estimated remaining time: 1 minute, 37 seconds.
Growing trees.. Progress: 66%. Estimated remaining time: 1 minute, 5 seconds.
Growing trees.. Progress: 82%. Estimated remaining time: 33 seconds.
Growing trees.. Progress: 99%. Estimated remaining time: 1 seconds.
Growing trees.. Progress: 16%. Estimated remaining time: 2 minutes, 42 seconds.
Growing trees.

[mbo] 0: num.trees=325; max.depth=6; min.node.size=514; mtry=35 : y = 2.23e+08 : 1025.7 secs : initdesign

[mbo] 0: num.trees=484; max.depth=24; min.node.size=463; mtry=14 : y = 2.44e+08 : 1884.4 secs : initdesign

[mbo] 0: num.trees=228; max.depth=12; min.node.size=360; mtry=47 : y = 2.34e+08 : 2064.7 secs : initdesign

[mbo] 0: num.trees=134; max.depth=9; min.node.size=733; mtry=34 : y = 2.33e+08 : 669.3 secs : initdesign

[mbo] 0: num.trees=310; max.depth=25; min.node.size=604; mtry=23 : y = 2.31e+08 : 1937.7 secs : initdesign

[mbo] 0: num.trees=287; max.depth=21; min.node.size=105; mtry=22 : y = 2.28e+08 : 1801.4 secs : initdesign

[mbo] 0: num.trees=179; max.depth=7; min.node.size=211; mtry=27 : y = 2.29e+08 : 556.9 secs : initdesign

[mbo] 0: num.trees=416; max.depth=1; min.node.size=166; mtry=7 : y = -1.4e+05 : 80.1 secs : initdesign

[mbo] 0: num.trees=43; max.depth=22; min.node.size=416; mtry=39 : y = 2.04e+08 : 487.1 secs : initdesign

[mbo] 0: num.trees=379; max.depth=18; m

20250902 035217	20	16	976	14	5	217840000	17


[mbo] 1: num.trees=20; max.depth=16; min.node.size=976; mtry=14 : y = 2.18e+08 : 81.8 secs : infill_ei

Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 894 points instead of 1000!”
Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 908 points instead of 1000!”
Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 936 points instead of 1000!”


...

[mbo] 3: num.trees=297; max.depth=12; min.node.size=328; mtry=21 : y = 2.39e+08 : 1293.3 secs : infill_ei

Saved the current state after iteration 4 in the file HT450.RDATA.

Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 792 points instead of 1000!”
Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 720 points instead of 1000!”


Growing trees.. Progress: 10%. Estimated remaining time: 4 minutes, 32 seconds.
Growing trees.. Progress: 21%. Estimated remaining time: 4 minutes, 2 seconds.
Growing trees.. Progress: 31%. Estimated remaining time: 3 minutes, 25 seconds.
Growing trees.. Progress: 42%. Estimated remaining time: 2 minutes, 52 seconds.
Growing trees.. Progress: 53%. Estimated remaining time: 2 minutes, 20 seconds.
Growing trees.. Progress: 63%. Estimated remaining time: 1 minute, 49 seconds.
Growing trees.. Progress: 73%. Estimated remaining time: 1 minute, 19 seconds.
Growing trees.. Progress: 83%. Estimated remaining time: 49 seconds.
Growing trees.. Progress: 94%. Estimated remaining time: 17 seconds.
Growing trees.. Progress: 10%. Estimated remaining time: 4 minutes, 39 seconds.
Growing trees.. Progress: 20%. Estimated remaining time: 4 minutes, 1 seconds.
Growing trees.. Progress: 31%. Estimated remaining time: 3 minutes, 27 seconds.
Growing trees.. Progress: 41%. Estimated remaining time: 2 minutes

[mbo] 4: num.trees=500; max.depth=30; min.node.size=307; mtry=11 : y = 2.44e+08 : 1636.5 secs : infill_ei

Saved the current state after iteration 5 in the file HT450.RDATA.

Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 396 points instead of 1000!”
Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 510 points instead of 1000!”


20250902 045140	20	7	1	50	5	214640000	21


[mbo] 5: num.trees=20; max.depth=7; min.node.size=1; mtry=50 : y = 2.15e+08 : 125.8 secs : infill_ei

Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 991 points instead of 1000!”
Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 807 points instead of 1000!”


In [ ]:
# analizo la salida de la bayesiana

tb_bayesiana <- fread(klog)
setorder( tb_bayesiana, -ganancia)
tb_bayesiana

In [ ]:
# mejores parametros

print( tb_bayesiana[1] )

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

Una vez que se impriman los mejores hiperparametros proceder:


1.   Copielos en el punto 4.04  en la celda donde se define PARAM
2.   Ejecute el punto 4.04 con estos hiperparámetros optimos
2.   Reporte el resultado en la hoja  **C4-RF Bayesiana**  de la  planilla colaborativa





---

